# Final Exam
Submitted by: Anthony Aromin

## Initial Setup

In [426]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

In [427]:
df = pd.read_csv('salary_dataset.csv')
df.head(5)

,Position,Annual Salary in Dollar,Date Hired
0,City Administrator,295000.0,2004.0
1,President,295000.0,2008.0
2,Superintendent Of Schools,275000.0,2007.0
3,Chief,230743.0,1990.0
4,Medical Officer Psych,206000.0,2008.0


In [428]:
df.columns = [
    'position',
    'annual_salary_usd',
    'date_hired'
]
df.head(5)

,position,annual_salary_usd,date_hired
0,City Administrator,295000.0,2004.0
1,President,295000.0,2008.0
2,Superintendent Of Schools,275000.0,2007.0
3,Chief,230743.0,1990.0
4,Medical Officer Psych,206000.0,2008.0


## Data Cleaning

In [429]:
df.dropna(subset=[
    'position',
    'date_hired'
],inplace=True)

df.isnull().sum()

position              0
annual_salary_usd    64
date_hired            0
dtype: int64

In [430]:
df.isnull().sum()

position              0
annual_salary_usd    64
date_hired            0
dtype: int64

In [431]:
df.describe()

,annual_salary_usd,date_hired
count,11434.000000,11498.000000
mean,88106.521077,1996.428596
std,25158.378394,10.094801
min,7.000000,1955.000000
25%,78743.000000,1988.000000
50%,85606.000000,1998.000000
75%,97296.000000,2006.000000
max,295000.000000,2011.000000


In [432]:
df['date_hired'] = df['date_hired'].astype(int)

df.head(5)

,position,annual_salary_usd,date_hired
0,City Administrator,295000.0,2004
1,President,295000.0,2008
2,Superintendent Of Schools,275000.0,2007
3,Chief,230743.0,1990
4,Medical Officer Psych,206000.0,2008


In [433]:
temp_df = df
temp_grouped_df = df.groupby('position')['annual_salary_usd']

def fill_annual_salary_usd(x):
    if (np.isnan(x['annual_salary_usd']) or
        x['annual_salary_usd'] < 1_000.00):
        mean_salary = temp_grouped_df.get_group(x['position']).mean()

        # if mean salary of position is still NaN or way less than minimum wage
        # use mean salary of all positions instead.
        if (np.isnan(mean_salary) or
            mean_salary < 1_000.00):
            mean_salary = df['annual_salary_usd'].mean()

        return mean_salary
    else:
        return x['annual_salary_usd']

df['annual_salary_usd'] = temp_df.apply(fill_annual_salary_usd, axis=1)

print(df.isnull().sum())
print('-' * 50)
print(df.describe())

position             0
annual_salary_usd    0
date_hired           0
dtype: int64
--------------------------------------------------
       annual_salary_usd    date_hired
count       11498.000000  11498.000000
mean        91586.769790   1996.428596
std         18200.788921     10.094801
min         71796.000000   1955.000000
25%         78743.000000   1988.000000
50%         87987.000000   1998.000000
75%         97674.250000   2006.000000
max        295000.000000   2011.000000


## Additional Columns

In [434]:
df['annual_salary_php'] = df.apply(
    lambda x: x['annual_salary_usd'] * 50.0, axis=1
)

df.head(5)

,position,annual_salary_usd,date_hired,annual_salary_php
0,City Administrator,295000.0,2004,14750000.0
1,President,295000.0,2008,14750000.0
2,Superintendent Of Schools,275000.0,2007,13750000.0
3,Chief,230743.0,1990,11537150.0
4,Medical Officer Psych,206000.0,2008,10300000.0


In [435]:
df['monthly_salary_php'] = df.apply(
    lambda x: np.round(x['annual_salary_php'] / 12.0, 2), axis=1
)

df.head(5)

,position,annual_salary_usd,date_hired,annual_salary_php,monthly_salary_php
0,City Administrator,295000.0,2004,14750000.0,1229166.67
1,President,295000.0,2008,14750000.0,1229166.67
2,Superintendent Of Schools,275000.0,2007,13750000.0,1145833.33
3,Chief,230743.0,1990,11537150.0,961429.17
4,Medical Officer Psych,206000.0,2008,10300000.0,858333.33


In [436]:
df['years_exp'] = df.apply(
    lambda x: 2015 - x['date_hired'], axis = 1
)

df.head(5)

,position,annual_salary_usd,date_hired,annual_salary_php,monthly_salary_php,years_exp
0,City Administrator,295000.0,2004,14750000.0,1229166.67,11
1,President,295000.0,2008,14750000.0,1229166.67,7
2,Superintendent Of Schools,275000.0,2007,13750000.0,1145833.33,8
3,Chief,230743.0,1990,11537150.0,961429.17,25
4,Medical Officer Psych,206000.0,2008,10300000.0,858333.33,7


## Exporting the Dataset

In [437]:
temp_df = df.groupby('position')

temp_df['years_exp'].count().describe()

count    1646.000000
mean        6.985419
std        55.818037
min         1.000000
25%         1.000000
50%         1.000000
75%         2.000000
max      1874.000000
Name: years_exp, dtype: float64

In [448]:
out_df = temp_df.filter(lambda x: len(x) >= 30)

out_df.groupby('position')['years_exp'].count().describe()

count      58.000000
mean      136.431034
std       268.209143
min        30.000000
25%        40.500000
50%        59.500000
75%       101.000000
max      1874.000000
Name: years_exp, dtype: float64

In [449]:
out_df.to_csv('salary_dataset_cleaned.csv')

##  
---

## Reimporting the Dataset

In [452]:
exp_df = pd.read_csv('salary_dataset_cleaned.csv')
exp_df.head(5)

,Unnamed: 0,position,annual_salary_usd,date_hired,annual_salary_php,monthly_salary_php,years_exp
0,9,Workers Compensation Recipient,197808.0,1984,9890400.0,824200.00,31
1,12,Chief Of Staff,193125.0,2011,9656250.0,804687.50,4
2,18,Medical Officer (psychiatry),188027.0,1987,9401350.0,783445.83,28
3,29,Deputy Director,182000.0,2011,9100000.0,758333.33,4
4,30,Medical Officer (psychiatry),180604.0,1989,9030200.0,752516.67,26


In [454]:
exp_df.drop([
    'Unnamed: 0',
    'annual_salary_usd',
    'date_hired',
    'annual_salary_php'
], axis=1, inplace=True)

exp_df.head(5)

,position,monthly_salary_php,years_exp
0,Workers Compensation Recipient,824200.00,31
1,Chief Of Staff,804687.50,4
2,Medical Officer (psychiatry),783445.83,28
3,Deputy Director,758333.33,4
4,Medical Officer (psychiatry),752516.67,26


## Training

In [399]:
from sklearn.model_selection import train_test_split

x = exp_df.drop('monthly_salary_php', axis=1)
y = exp_df['monthly_salary_php']
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=1/3)

In [400]:
from sklearn.linear_model import LinearRegression

model = LinearRegression()
model.fit(x_train, y_train)

ValueError: could not convert string to float: 'Detective'

## Testing